# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract image data 

(4000, 128, 128)


## Read dataframe

In [6]:
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/',
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [34]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200811_195351_bsize256_8gpurun_noconvbrelu', '2020…

In [50]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20200812_122250_bsize64_8gpurun


In [51]:
# main_dir=''

In [52]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)

In [53]:
print(df.shape)
df.head()


(70320, 19)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
0,0,0,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,189.955298,4.778167,0.013186,194.746651,12.417616,21.439721,18236.349774,3.549975e+07,0,0,"[0.00021028518730998038, 0.058279842228064906,...","[4.0617537070536625e-05, 0.001978581126841826,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[56690757.93387408, 2306.3432735726237, 1060.8...","[41895.0660429794, 69.45253299107003, 36.85561..."
1,0,1,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,189.299435,5.986939,0.013186,195.299560,12.170548,22.906236,18166.039840,3.533201e+07,0,0,"[0.0003285706051718441, 0.06979832063936273, 0...","[4.1581796383402746e-05, 0.00241766216573588, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[56828063.689699516, 2173.999922736897, 980.86...","[41863.89988651487, 89.12905592354626, 33.4052..."
2,0,2,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,174.167120,4.884275,0.013186,179.064581,11.711918,22.069416,38341.739858,7.037632e+07,0,0,"[0.00022342801151685416, 0.07374894630348355, ...","[2.757721051645845e-05, 0.0027596312030048952,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[58538393.57175751, 734.243693731376, 644.2099...","[24734.127336607675, 31.414590784839678, 23.68..."
3,0,3,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,154.865966,3.972353,0.013186,158.851505,10.996951,17.844643,33427.960204,5.393823e+07,0,0,"[0.001888811593159288, 0.10478746907163233, 0....","[0.00011042362913049002, 0.0029292996190210256...","[-1.031746031584782, -0.6161616169043975, -0.3...","[60962737.393075265, 931.9592139590532, 668.45...","[35489.20099080732, 39.76956336232921, 21.9654..."
4,0,4,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,137.862341,3.210343,0.013186,141.085870,10.255475,17.590514,14864.946822,2.876259e+07,0,0,"[0.005602598204758762, 0.12526050229294267, 0....","[0.0002899517576741754, 0.0032448837584338297,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[63701318.4964969, 4126.343995176954, 1452.868...","[84315.94505243078, 109.72321859465141, 43.014..."


### View best epochs

#### Locations with best chi_sqr

In [54]:
chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]

,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
48346,13,48346,0.000144,0.002611,0.038764,0.041519,0.000077,11.072417,0.675964,4.184022e+03
27346,7,27346,0.005468,0.000554,0.108728,0.114750,0.004809,13.762911,0.284416,8.646092e+02
69512,19,69512,0.028454,0.005609,0.000884,0.034947,0.018650,11.879335,0.570693,3.304405e+03
24258,6,24258,0.000811,0.001638,0.001451,0.003899,0.000276,10.513582,0.417502,2.356353e+03
66878,19,66878,0.000354,0.001198,0.002991,0.004543,0.000041,11.122017,2.003997,1.261267e+04
32,0,32,4.047297,0.294280,0.013186,4.354763,2.061429,2.012612,9576.130739,7.916819e+07
28864,8,28864,0.036062,0.002523,0.040729,0.079314,0.025325,12.443831,0.052046,3.464173e+02
28865,8,28865,0.042864,0.002510,0.084497,0.129870,0.027547,15.425425,0.057497,2.440007e+02


In [55]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

In [56]:
df_sliced=f_get_best_chisqr_models(df,cutoff=0.3)
print(df_sliced.shape)

(6485, 19)


In [57]:
df_sliced.sort_values(by=['chi_1','chi_spec1'])[['epoch','step','chi_1','chi_spec1']].head(5)

,epoch,step,chi_1,chi_spec1
24258,6,24258,0.003899,0.417502
23967,6,23967,0.004480,0.245749
22586,6,22586,0.004614,0.211245
21219,6,21219,0.004738,0.418183
19643,5,19643,0.004817,0.200981


In [58]:
### Plot chi-sqr values
df_sliced.plot(x="epoch", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### High Pixel images

In [59]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

In [60]:
df[df.num_vlarge>10].tail()

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
912,0,912,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.606931,0.880546,105.717485,107.204962,0.383721,28.340413,7.407881,24875.981836,49,17,"[1.4029531665059167, 0.44564003145983305, 0.21...","[0.009903431022406877, 0.007076517838726303, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[210129919.52979922, 95126.4355116114, 73327.6...","[927623.0448125026, 4979.446173933056, 2259.01..."
913,0,913,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.604936,0.751726,51.853420,53.210082,0.371017,26.822081,7.453176,19974.345137,39,13,"[1.3997847739390463, 0.4474986796995289, 0.223...","[0.01200833093283437, 0.008446657516295263, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[210140477.35273618, 87931.7002068013, 68977.1...","[1113859.6798039572, 4097.3997526985795, 2169...."
915,0,915,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.505282,0.505648,35.816428,36.827357,0.347608,24.059338,6.768756,21825.317333,38,13,"[1.4005300587782201, 0.4611741112937677, 0.225...","[0.011844294029221101, 0.008787999777702796, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[211267900.9596249, 85584.57375454903, 67020.6...","[1065779.7462966982, 3711.005516858218, 2268.8..."
955,0,955,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.335347,0.100411,14.036235,14.471994,0.305503,18.909625,6.909001,41808.508508,41,12,"[1.4041448417172842, 0.4927614000968617, 0.227...","[0.011319857115230566, 0.00927538371739659, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[213969464.13819265, 117690.51646345854, 86805...","[944886.750554567, 4024.2622770935022, 2844.59..."
1200,0,1200,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.677417,3.853021,172.808921,177.339360,0.445262,47.761677,24.655586,64914.331235,40,12,"[1.4238987904544287, 0.4473461525323548, 0.179...","[0.01589828769800353, 0.008828260306245578, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[208549911.21496356, 384860.3101103902, 283869...","[1754391.7838603475, 27931.309803653854, 19343..."


## Compare samples

In [61]:
# df_sliced.sort_values(by=['chi_spec1','chi_1']).head(5)
# df_sliced.sort_values(by=['chi_1','chi_spec1']).head(5)

In [62]:
best_idx=[]
best_idx.append(df_sliced.sort_values(by=['chi_1','chi_spec1']).head(2).index)
best_idx.append(df_sliced.sort_values(by=['chi_spec1','chi_1']).head(2).index)

best_idx=[i for j in best_idx for i in j]
print(best_idx)

df_best=df_sliced.loc[best_idx]
print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])
df_best

[24258, 23967, 32897, 20355]
[(6, 24258), (6, 23967), (9, 32897), (5, 20355)]


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
24258,6,24258,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.000811,0.001638,0.001451,0.003899,0.000276,10.513582,0.417502,2356.353493,0,0,"[1.1083869366624928, 0.9693194940716503, 0.266...","[0.007997891993363362, 0.006022409817427079, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[197215018.05779856, 82798.28693154454, 61906....","[600416.1656066332, 4392.5186541644725, 2602.6..."
23967,6,23967,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.001191,0.001636,0.001653,0.004480,0.000642,10.332230,0.245749,1329.662773,0,0,"[1.1025515227146405, 0.9763267651338512, 0.268...","[0.008106219173980078, 0.006013930097842057, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[197152448.12628508, 78960.41604020819, 57515....","[616827.7733313359, 5127.348421762532, 2665.24..."
32897,9,32897,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.012291,0.002870,0.004275,0.019436,0.005249,13.169306,0.091701,404.233285,2,0,"[1.0694916871954072, 0.9972925201519564, 0.291...","[0.01142841581893856, 0.0078025445518599985, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[194957240.39434028, 88857.21571308374, 61552....","[844592.3825573911, 7782.35193310601, 3425.241..."
20355,5,20355,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.017750,0.010653,0.003093,0.031496,0.010196,12.569058,0.091769,433.703915,1,0,"[1.0522952404938706, 1.0233507172135805, 0.294...","[0.011726530446497473, 0.008729392287280799, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[194537789.49222988, 84887.15266721696, 58558....","[836542.0212451399, 6774.197119777208, 3039.31..."


## Plot pixel intensity and spectrum

In [63]:
def f_plot_hist_spec_combined(df,dict_bkg):

    fig=plt.figure(figsize=(6,6))
    ax1=fig.add_subplot(121)
    ax2=fig.add_subplot(122)
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

        x1=row.hist_bin_centers
        y1=row.hist_val
        yerr1=row.hist_err
        x1=f_invtransform(x1)

        y2=row.spec_val
        yerr2=row.spec_err
        x2=np.arange(len(y2))

        label='{0}_{1}_{2}'.format(i,row.epoch,row.step)
        ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
    #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

        ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
        ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
    x=f_invtransform(x)
    ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   

    y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
    x=np.arange(len(y))
    ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)

    plt.legend()
    # plt.yscale('log')
    ax1.set_xscale('symlog',linthreshx=50)
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    
    


In [48]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[:4000][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)


In [49]:
f_plot_hist_spec_combined(df_best,dict_val)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## View image block

In [ ]:
def f_get_img(df,epoch,step):
    df_temp=df[(df.epoch==epoch)&(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,17,14207)

In [ ]:
f_plot_grid(img_arr[:18],cols=6,fig_size=(10,6))

In [ ]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,6))

In [ ]:
df_sliced[df_sliced.epoch==4]['step'].values

In [ ]:
df[(df.epoch==18)&(df.step>=15975)&(df.step<=15983)][['epoch','step','chi_1','chi_spec1','num_vlarge']]